![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9b/Carl_Friedrich_Gauss.jpg/330px-Carl_Friedrich_Gauss.jpg)

# Описание метода

Даны матрица $A$ размером $N\times N$, вектор $B$ высотой $N$. Требуется найти вектор $X$ такой, что $AX=B$.

Без терминологии линейной алгебры это записывается, как:

$
\begin{array}{lcr}
a_{11}x_1+\ldots+a_{1N}x_N &=& b_1 \\
\ldots & & \\
a_{N1}x_1+\ldots+a_{NN}x_N &=& b_N \\
\end{array}
$

## Прямой ход

Из второго уравнения вычитаем первое, умноженное на $\frac{a_{21}}{a_{11}}$. Из третьего вычитаем первое, умноженное на $\frac{a_{31}}{a_{11}}$. И т.д. до $N$-го. Получаем систему уравнений следующего вида.

$
\begin{array}{lcr}
a_{11}&x_1 +& a_{12} x_2 +\ldots+a_{1N}x_N &=& b_1 \\
0 &x_1 +& a'_{22} x_2 +\ldots+a'_{1N}x_N &=& b'_1 \\
& \ldots & & \\
0 &x_1 +& a'_{N2} x_2 +\ldots+a'_{NN}x_N &=& b'_N \\
\end{array}
$

Затем из третьего уравнения вычитаем второе, умноженное на $\frac{a_{32}}{a_{22}}$.

И т.д., пока не выполним такое же действие над последней строкой.

В итоге мы получим систему уравнений следующего вида.

$
\begin{array}{lcr}
a_{11}& x_1 +& a_{12}   & x_2 + & a''_{13} & x_3 + \ldots + a_{1N}   x_N &=& b_1 \\
0     & x_1 +& a''_{22} & x_2 + & a''_{23} & x_3 + \ldots + a''_{2N} x_N &=& b''_1 \\
0     & x_1 +& 0        & x_2 + & a''_{33} & x_3  +\ldots + a''_{3N} x_N &=& b''_3 \\
0     & x_1 +& 0        & x_2 + & 0        & x_3 + \ldots + a''_{4N} x_N &=& b''_4 \\
& & & \ldots & & & & \\
0 & x_1 +& & \ldots & +                  & 0 x_{N-1} + a''_{NN} x_N &=& b''_N \\
\end{array}
$


## Обратный ход

Очевидно, $x_N = \frac{b''_N}{a''_{NN}}$.

Далее, $x_i = \frac{b''_i - \sum_{j=i+1}^{N} a''_{ij}x_j}{a_{ii}}$

# Прямой и обратный ход в вектроной форме

Интересно? Тогда см. ниже. Неинтересно? Тоже см. ниже.

# Задание

Предлагается реализовать слкдующие недостающие функции

In [22]:
from numpy.linalg import norm
from numpy.linalg import solve as solve_out_of_the_box
from numpy import array

def gauss(a, b):
    a = a.copy()
    b = b.copy()

    def forward():
        # do something to a and b
        ...

    def backward():
        x = np.zeros(len(b), dtype=float)
        # then do something to a and b
        ...
        return x

    forward()
    x = backward()
    return x

a = array([
    [1.5, 2.0, 1.5, 2.0],
    [3.0, 2.0, 4.0, 1.0],
    [1.0, 6.0, 0.0, 4  ],
    [2.0, 1.0, 4.0, 3  ]
], dtype=float)

b = array([5, 6, 7, 8], dtype=float)

oob_solution = solve_out_of_the_box(a, b)
solution = gauss(ab)

print(solution)
print("Макс отклонение компоненты решения:", norm(solution-oob_solution, ord=1))

[0.8490566  0.05660377 0.47169811 1.45283019]
Макс отклонение компоненты решения: 0.0


# Прямой и обратный ход в векторной форме

Очевидно, что на прямом ходе мы оперируем целиком уравнениями (включая свободные члены): умножаем всё уравнение на число, вычитаем одно уравнение из другого и т.д., т.е. оперируем целыми уравнениями, (строками матрицы), а не отдельными коэффициентами. При вычислениях это приводит к избыточности, но в случае, если компьютер может выполнять векторные вычисления (т.е. обрабатывать строку матрицы, как одно число), это даёт большое ускорение.

Сразу скажем, что реализация NumPy «из коробки» так не умеет. «Убедить» её это делать, пусть и не «из коробки», можно, но это не тема данного блокнота.

Итак, пусть у нас $d$ уравнений. $\mathrm{AB}$ — матрица из $d$ строк и $d+1$ столбцов, у которой правый слобец состоит из свободных членов уравнений. Запишем алгоритм в «императивной» форме.

$ab_i$ — строка, $ab_{i,j}$ — элемент матрицы.

## Прямой ход


$ab_1 \gets \frac{ ab_1 }{ ab_{1,1}}$ — делаем 1 в левом верхнем углу

Далее для строк c индексами $j \in [2, d]$: $ab_j \gets ab_j - ab_1 * ab_{j,1}$ — вычитаем первую строку (у неё уже 1 в начале), умноженную на первый элемент $j$-й строки, делая первый элемент равным 0.

$ab_2 \gets ab_2 / ab_{2, 2}$ — делаем $1$ «почти» в левом верхнем углу.

и т.д.

В итоге прямого хода получаем $\mathrm{AB}$ с единицами на главной диагонали $A$ и нулями ниже её.

## Обратный ход

Для каждого $i$-го уравнения повторяем следующее.

$ab_i \gets ab_i - ab_{i+1} * ab_{i, i + 1} - \ldots - ab_{d} * ab_{i, d}$.

## Результат

По итогам обратного хода получем $\mathrm{AB}$ такую, что $A$ — единичная матрица, а $B$ — решение системы.

In [21]:
from numpy.linalg import norm
from numpy import array

ab = array([
    [1.5, 2.0, 1.5, 2.0, 5.0],
    [3.0, 2.0, 4.0, 1.0, 6.0],
    [1.0, 6.0, 0.0, 4  , 7.0],
    [2.0, 1.0, 4.0, 3  , 8.0]
], dtype=float)

def vector_gauss(ab):
    ab = ab.copy()
    d = len(ab) # размер по старшему измерению

    # прямой
    for i in range(d):
        ab_ii = ab[i, i]
        ...
    ...
    # запрограмируйте!

    # обратный
    for i in range(d - 1, -1, -1):
        ...
    ...
    # запрограмируйте!
    
    x = ab[:, -1]  # Последний столбец
    return x

solution = vector_gauss(ab)
oob_solution = solve_out_of_the_box(a, b)

print(solution)
print("Макс отклонение компоненты решения:", norm(solution-oob_solution, ord=1))

[0.8490566  0.05660377 0.47169811 1.45283019]
Макс отклонение компоненты решения: 3.823330541052883e-15
